In [57]:
import sys
sys.path.append('..')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem

# MAO-B (monoamine oxidase B) - its inhibitors can be used to treat symptoms of Parkinson's disease

In [69]:
import csv
import pandas as pd
pd.set_option('display.max_columns', None)
with open('aktywnosc.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    df = pd.read_csv(csvfile, sep=';')

In [70]:
df.head()

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,Standard Value,Standard Units,pChEMBL Value,Data Validity Comment,Comment,Uo Units,Ligand Efficiency BEI,Ligand Efficiency LE,Ligand Efficiency LLE,Ligand Efficiency SEI,Potential Duplicate,Assay ChEMBL ID,Assay Description,Assay Type,BAO Format ID,BAO Label,Assay Organism,Assay Tissue ChEMBL ID,Assay Tissue Name,Assay Cell Type,Assay Subcellular Fraction,Assay Parameters,Assay Variant Accession,Assay Variant Mutation,Target ChEMBL ID,Target Name,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties
0,CHEMBL4205890,NaN,None,333.39,0,3.31,4a,C#CCN(CCOc1ccc2ccc(=O)oc2c1)Cc1ccccc1,IC50,'=',8298.0,nM,5.08,NaN,NaN,UO_0000065,15.24,0.28,1.77,11.90,0,CHEMBL4192761,Inhibition of recombinant human MAO-B using p-...,B,BAO_0000357,single protein format,Homo sapiens,None,None,None,None,NaN,NaN,NaN,CHEMBL2039,Monoamine oxidase B,Homo sapiens,SINGLE PROTEIN,CHEMBL4190276,1,Scientific Literature,Eur J Med Chem,2017.0,None,NaN
1,CHEMBL4216262,NaN,None,375.47,0,4.40,4p,C#CCN(CCCCOc1ccc2ccc(=O)oc2c1)Cc1ccc(C)cc1,IC50,'=',1676.0,nM,5.78,NaN,NaN,UO_0000065,15.38,0.28,1.38,13.53,0,CHEMBL4192761,Inhibition of recombinant human MAO-B using p-...,B,BAO_0000357,single protein format,Homo sapiens,None,None,None,None,NaN,NaN,NaN,CHEMBL2039,Monoamine oxidase B,Homo sapiens,SINGLE PROTEIN,CHEMBL4190276,1,Scientific Literature,Eur J Med Chem,2017.0,None,NaN
2,CHEMBL341071,NaN,None,196.21,0,2.00,10,Cc1cc2c(nn1)-c1ccccc1C2=O,IC50,'=',130.0,nM,6.89,NaN,NaN,UO_0000065,35.10,0.63,4.89,16.07,0,CHEMBL734178,Inhibitory concentration against Monoamine oxi...,B,BAO_0000357,single protein format,Papio hamadryas,None,None,None,None,NaN,NaN,NaN,CHEMBL2039,Monoamine oxidase B,Homo sapiens,SINGLE PROTEIN,CHEMBL1136363,1,Scientific Literature,Bioorg Med Chem Lett,2003.0,None,NaN
3,CHEMBL4111335,NaN,None,337.47,0,3.38,BDBM254597,NC1CCC(N[C@H]2C[C@@H]2c2ccc(OCc3cccnc3)cc2)CC1,IC50,'>',50000.0,nM,NaN,NaN,443212,UO_0000065,NaN,NaN,NaN,NaN,0,CHEMBL3887028,Fluorescence-Based (Inhibitor)-Screening Assay...,B,BAO_0000357,single protein format,Homo sapiens,None,None,None,None,NaN,NaN,NaN,CHEMBL2039,Monoamine oxidase B,Homo sapiens,SINGLE PROTEIN,CHEMBL3886143,37,BindingDB Database,NaN,2016.0,None,NaN
4,CHEMBL4474002,NaN,None,353.80,0,1.71,25,Cc1cc(O)cc2oc(=O)c(C(=O)CCN3CCOCC3)cc12.Cl,IC50,'=',35980.0,nM,4.44,NaN,NaN,UO_0000065,14.00,0.26,2.73,5.56,0,CHEMBL4349201,Inhibition of recombinant human MAO-B expresse...,B,BAO_0000219,cell-based format,Homo sapiens,None,None,None,None,NaN,NaN,NaN,CHEMBL2039,Monoamine oxidase B,Homo sapiens,SINGLE PROTEIN,CHEMBL4346711,1,Scientific Literature,Eur J Med Chem,2019.0,None,NaN


In [71]:
activity_type = 'IC50'  # TODO: You can change this
relevant_columns = ['Smiles', 'Standard Value']
df = df[(df['Standard Type'] == activity_type) & (df['Standard Relation'].str.contains('='))][relevant_columns]
df

,Smiles,Standard Value
0,C#CCN(CCOc1ccc2ccc(=O)oc2c1)Cc1ccccc1,8298.00
1,C#CCN(CCCCOc1ccc2ccc(=O)oc2c1)Cc1ccc(C)cc1,1676.00
2,Cc1cc2c(nn1)-c1ccccc1C2=O,130.00
4,Cc1cc(O)cc2oc(=O)c(C(=O)CCN3CCOCC3)cc12.Cl,35980.00
5,Cc1cc(O)cc2oc(=O)c(C(=O)CCN3CCCC3)cc12.Cl,43670.00
...,...,...
5304,O=C1c2ccccc2-c2nnc(-c3ccccc3)cc21,177.83
5305,C#CCN(C)C1=C(Cl)C(=O)c2ccccc2C1=O,140.00
5308,O=C1/C(=C/c2ccc[nH]2)COc2ccccc21,13760.00
5312,O=c1c2ccc(Cl)cc2oc2ccc(OCCCCCCN3CCCCC3)cc12,946.00


In [72]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)
len(df_train), len(df_test)
df_train, df_val = train_test_split(df_train, test_size=0.1)
len(df_train), len(df_val), len(df_test)

(3178, 354, 883)

In [82]:
#definiujemy klasy służące do ekstracji cech z danych 

class Featurizer: #Definiuje interfejs dla obiektów, które generują cechy z danych.
    
    def __init__(self, y_column, smiles_col='Smiles', **kwargs): #y_column- nazwa kolumny z y, a smiles_col- nazwa kolumny ze SMILES
                                                               # **kwargs- przechowywanie innych argumentów przekazywanych do metody
        self.y_column = y_column           
        self.smiles_col = smiles_col       #Dzięki self można odwołać się do atrybutów i metod tej konkretnej instancji wewnątrz klasy.
        self.__dict__.update(kwargs)
    
    
    def __call__(self, df):                #W tym przypadku metoda __call__ nie jest zdefiniowana, co oznacza, 
        raise NotImplementedError()        #że każda klasa dziedzicząca po Featurizer musi ją zaimplementować, aby można było ją wywołać. 


class CheatingFeaturizer(Featurizer):      #dziedziczy po klasie Featurizer
    
    
    def __call__(self, df):                # W metodzie __call__() tej klasy, dla każdego wiersza w danych wejściowych df, 
        
        predictions = []                   
        labels = []
        predictions_column_name = 'Standard Value'                
        
        for i, row in df.iterrows():
            predictions.append(row[predictions_column_name])      # Wartość predykcji pobierana jest z kolumny 'Y',
            labels.append(row[self.y_column])                     # Wartość etykiety z kolumny zdefiniowanej w klasie bazowej y_column.
            
        predictions = np.array(predictions)       #wartości etykiet i predykcji są zwracane w postaci tablic numpy
        labels = np.array(labels)
        return predictions, labels
        
        

class ECFPFeaturizer(Featurizer):               #też dziedziczy po Featurizer
    
    def __init__(self, y_column, radius=2, length=1024, **kwargs):
        
        self.radius = radius                    #ustawiamy wartości pól radius i length w obiekcie ECFPFeaturizer na wartości podane przy tworzeniu obiektu.
        self.length = length
        super().__init__(y_column, **kwargs)    #wywołanie konstruktora klasy nadrzędnej Featurizer 
        
        #Dzięki temu, do pola y_column w obiekcie ECFPFeaturizer zostaje przypisana wartość argumentu y_column 
        #podanego przy tworzeniu obiektu, a wszystkie dodatkowe argumenty kwargs zostaną zapisane w formie słownikowej pod atrybutami obiektu ECFPFeaturizer.
        
        
        
    def __call__(self, df):  #W tej metodzie obliczane są fingerprinty dla cząsteczek na podstawie SMILES-ów, 
                             #które znajdują się w kolumnie określonej przez argument smiles_col
        fingerprints = []
        labels = []
        for i, row in df.iterrows():
            y = row[self.y_column]
            smiles = row[self.smiles_col]
            mol = Chem.MolFromSmiles(smiles)
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, self.radius, nBits=self.length) #Fingerprinty obliczane są z użyciem funkcji GetMorganFingerprintAsBitVect z modułu AllChem biblioteki RDKit, która tworzy reprezentację molekularną cząsteczki.
            fingerprints.append(fp)
            labels.append(y)
            
        fingerprints = np.array(fingerprints)
        labels = np.array(labels)
        return fingerprints, labels

In [83]:
# TODO: Use your training script
featurizer = ECFPFeaturizer(y_column='Standard Value')
scores = []
X_train, y_train = featurizer(df_train)
X_test, y_test = featurizer(df_test)


In [84]:
from sklearn.ensemble import RandomForestRegressor

def train(X_train, y_train, X_valid, y_valid):
    # create and train your model 
    model = RandomForestRegressor(verbose=1)
    model.fit(X_train, y_train)
    return model

In [85]:
def predict(model, X_test, y_test):
    y_pred = model.predict(X_test)
    return y_pred

In [86]:
X_train, y_train = featurizer(df_train)
X_valid, y_valid = featurizer(df_val)
X_test, y_test = featurizer(df_test)
model = train(X_train, y_train, X_valid, y_valid)
predictions = predict(model, X_test, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


In [87]:
print(predictions[:10])

[2.19680000e+03 5.88754690e+03 9.18693938e+02 6.98806133e+00
 5.27408710e+03 8.56290954e+02 3.83018070e+04 6.36136885e+03
 5.43068000e+03 1.89188023e+03]


In [88]:
from tdc import Evaluator

In [89]:
rmse = Evaluator(name = 'RMSE')
mae = Evaluator(name = 'MAE')
r_squared = Evaluator(name = 'R2')
rmse_score = rmse(y_test, predictions)
mae_score = mae(y_test, predictions)
r2_score = r_squared(y_test, predictions)
print(f'RMSE, MAE, R2 = {rmse_score}, {mae_score}, {r2_score}')

RMSE, MAE, R2 = 2121660.1941118287, 142566.3431336471, -0.0041320703450289376
